# Preprocessing for ESTDA

## Loading data, preprocessing and feature engineering for ESTDA purposes

### Load data

In [1]:
import numpy as np
import pandas as pd
import polars as pl

import matplotlib.pyplot as plt

import folium
from mpl_toolkits.basemap import Basemap

%matplotlib inline

In [2]:
import xarray as xr
import glob

def preprocess_dataset(ds: xr.Dataset) -> xr.Dataset:
    """
    Preprocess the dataset to ensure the `valid_time` coordinate is in `datetime64[ns]` format.

    Args:
        ds (xr.Dataset): Input dataset.

    Returns:
        xr.Dataset: Preprocessed dataset with `valid_time` converted to datetime.
    """
    if "valid_time" in ds.coords:
        ds["valid_time"] = ds["valid_time"].dt.floor("D")  # Truncate to date only
    return ds

# Define the directory containing your NetCDF files
data_dir = 'drought-forecasting/modeling-pipeline/data/01_raw/unzipped_files/ERA5_monthly_averaged_data_on_single_levels/'

# Create a sorted list of file paths
file_list = sorted(glob.glob(data_dir + '*.nc'))

# Open multiple files as a single dataset with preprocessing applied
ds = xr.open_mfdataset(
    file_list, 
    combine='by_coords', 
    preprocess=preprocess_dataset
)


In [3]:
ds

<xarray.Dataset> Size: 55GB
Dimensions:     (valid_time: 551, latitude: 721, longitude: 1440)
Coordinates:
    number      int64 8B 0
  * valid_time  (valid_time) datetime64[ns] 4kB 1979-01-01 ... 2024-11-01
  * latitude    (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude   (longitude) float64 12kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
    expver      (valid_time) <U4 9kB dask.array<chunksize=(551,), meta=np.ndarray>
Data variables: (12/24)
    cl          (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(92, 121, 240), meta=np.ndarray>
    cvh         (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(92, 121, 240), meta=np.ndarray>
    tvh         (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(92, 121, 240), meta=np.ndarray>
    asn         (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(92, 121, 240), meta=np.ndarray>
    sst         (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(92, 121, 240), meta=np.ndarray>
    slt         (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(92, 121, 240), meta=np.ndarray>
    ...          ...
    si10        (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(92, 121, 240), meta=np.ndarray>
    e           (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(92, 121, 240), meta=np.ndarray>
    ro          (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(92, 121, 240), meta=np.ndarray>
    tp          (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(92, 121, 240), meta=np.ndarray>
    pev         (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(92, 121, 240), meta=np.ndarray>
    swvl1       (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(92, 121, 240), meta=np.ndarray>
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts

### Data Preprocessing 

#### Missing values

In [4]:
import xarray as xr

def check_missing_percentage(dataset: xr.Dataset) -> dict:
    """
    Calculate the percentage of missing values for each variable in the xarray.Dataset.

    Args:
        dataset (xr.Dataset): The input dataset.

    Returns:
        dict: A dictionary with variable names as keys and their respective missing percentage as values.
    """
    missing_percentages = {}
    for var in dataset.data_vars:
        data = dataset[var]
        # Calculate the number of missing values and total elements
        missing_count = data.isnull().sum().item()
        total_count = data.size
        # Calculate the missing percentage
        missing_percentage = (missing_count / total_count) * 100
        missing_percentages[var] = missing_percentage
    
    return missing_percentages


In [5]:
# # Example usage
# missing_perc = check_missing_percentage(ds)

In [6]:
import xarray as xr
from typing import List, Union

def fillna_in_variables(
    ds: xr.Dataset,
    variables: List[str],
    fill_value: Union[int, float]
) -> xr.Dataset:
    """
    Fills NaN values in specified variables of an xarray.Dataset with a provided value.

    Args:
        ds (xr.Dataset): The input dataset containing the data variables.
        variables (List[str]): A list of variable names in the dataset for which to fill NaN values.
        fill_value (Union[int, float]): The value to fill NaN values with (e.g., 0).

    Returns:
        xr.Dataset: A new dataset with NaN values filled in the specified variables.
    
    Example:
        ds = fillna_in_variables(ds, ["sst", "t2m"], fill_value=0)
    """
    # Check if each variable exists in the dataset
    for var in variables:
        if var not in ds:
            raise ValueError(f"Variable '{var}' not found in the dataset.")
    
    # Fill NaN values for the specified variables
    filled_ds = ds.copy()
    for var in variables:
        filled_ds[var] = filled_ds[var].fillna(fill_value)
    
    return filled_ds

In [7]:
ds = fillna_in_variables(ds, ["sst"], fill_value=0)

In [8]:
# # Example usage
# missing_perc = check_missing_percentage(ds)

In [9]:
import numpy as np
import xarray as xr

def interpolate_to_target_grid(ds, target_lat, target_lon, method="linear"):
    """
    Interpolates an xarray Dataset to a specified target latitude and longitude grid.
    
    Parameters:
        ds (xr.Dataset): The input dataset to be interpolated.
        target_lat (array-like): Target latitude values (e.g., np.arange(-90, 90, 1)).
        target_lon (array-like): Target longitude values (e.g., np.arange(0, 360, 1)).
        method (str): Interpolation method. Defaults to 'linear'.
                      Options include 'linear', 'nearest', etc.
    
    Returns:
        xr.Dataset: The interpolated dataset on the specified target grid.
    """
    # Step 1: Ensure latitude is in ascending order
    if ds.latitude[0] > ds.latitude[-1]:
        ds = ds.sortby("latitude")
    
    # Step 2: Perform interpolation
    interpolated_ds = ds.interp(latitude=target_lat, longitude=target_lon, method=method)
    
    return interpolated_ds

# Example usage
target_lat = np.arange(-90, 90, 1)   # 1° resolution latitude
target_lon = np.arange(0, 360, 1)   # 1° resolution longitude
method = "linear"

# OPTIONAL
# ds = interpolate_to_target_grid(ds, target_lat, target_lon, method)

### Feature Engineering 

In [10]:
# Update swvl1: Set to 1 wherever lsm indicates water 
ds['swvl1'] = ds['swvl1'].where(ds['lsm'] != 0, other=1)

# Update pev: Set to 0 wherever lsm indicates water 
ds['pev'] = ds['pev'].where(ds['lsm'] != 0, other=0)

In [11]:
# Save the original attributes
original_attrs = ds['tp'].attrs

# Perform the conversion
ds['tp'] = ds['tp'] * 1000

# Restore the original attributes and update the units
ds['tp'].attrs = original_attrs
ds['tp'].attrs['units'] = 'mm'  # Update the units attribute to 'mm'

In [12]:
# Save the original attributes
original_attrs = ds['pev'].attrs

# Perform the conversion
ds['pev'] = ds['pev'] * 1000

# Restore the original attributes and update the units
ds['pev'].attrs = original_attrs
ds['pev'].attrs['units'] = 'mm'  # Update the units attribute to 'mm'

In [13]:
# Save the original attributes
original_attrs = ds['e'].attrs

# Perform the conversion
ds['e'] = ds['e'] * 1000

# Restore the original attributes and update the units
ds['e'].attrs = original_attrs
ds['e'].attrs['units'] = 'mm'  # Update the units attribute to 'mm'

### Additional variables for deeper exploration
Target variables:
- tp 
- t2m 
- swvl1
- pev 

#### Percent of climatology mean

In [14]:
import xarray as xr
import numpy as np
from typing import Tuple

def calculate_percent_of_normal(
    ds: xr.Dataset, 
    variable: str, 
    date_range: Tuple[str, str]
) -> xr.Dataset:
    """
    Generalized function to calculate Percent of Normal for a given variable.

    Args:
        ds (xr.Dataset): Input dataset containing the variable.
        variable (str): Variable name for which Percent of Normal is to be calculated.
        date_range (Tuple[str, str]): Climatology period as (start_date, end_date).

    Returns:
        xr.Dataset: Dataset with Percent of Normal added as a new variable.
    """
    # Ensure the variable exists
    if variable not in ds.variables:
        raise ValueError(f"Variable '{variable}' not found in the dataset.")

    # Step 1: Filter dataset for the climatology period
    start_date, end_date = date_range
    ds_clim_period = ds.sel(valid_time=slice(start_date, end_date))

    # Step 2: Extract the variable and calculate monthly climatology mean
    data_clim_period = ds_clim_period[variable]
    climatology_mean = data_clim_period.groupby('valid_time.month').mean('valid_time')

    # Replace zeros and NaNs in climatology_mean with a small number
    climatology_mean = climatology_mean.where(climatology_mean > 0, other=1e-10).fillna(1e-10)

    # Step 3: Calculate Percent of Normal for all valid_time in the original dataset
    data_full = ds[variable]
    percent_of_normal = xr.apply_ufunc(
        lambda x, m: (x / m) * 100,
        data_full.groupby('valid_time.month'),
        climatology_mean,
        dask="allowed"
    )
    
    # Step 4: Add Percent of Normal to the original dataset
    ds_result = ds.copy()
    ds_result[f"{variable}_percent_of_normal"] = percent_of_normal

    return ds_result


In [15]:
date_range = ("1991-01-01", "2020-12-31")

# Assuming `ds` is your dataset
ds = calculate_percent_of_normal(ds, variable='tp', date_range=date_range)
ds = calculate_percent_of_normal(ds, variable='t2m', date_range=date_range)
ds = calculate_percent_of_normal(ds, variable='swvl1', date_range=date_range)

#### Standardized anomalies using standard deviation

In [16]:
import xarray as xr
import numpy as np
from typing import Tuple

def calculate_standardized_anomalies(
    ds: xr.Dataset, 
    variable: str, 
    date_range: Tuple[str, str]
) -> xr.Dataset:
    """
    Calculate standardized monthly anomalies for a specific variable,
    handling cases with zero standard deviation or NaNs.
    """
    # Ensure the variable exists
    if variable not in ds.variables:
        raise ValueError(f"Variable '{variable}' not found in the dataset.")
    
    # Step 1: Filter dataset for the climatology period
    start_date, end_date = date_range
    ds_clim_period = ds.sel(valid_time=slice(start_date, end_date))
    
    # Step 2: Extract the variable and calculate monthly climatology (mean and std)
    data_clim_period = ds_clim_period[variable]
    climatology_mean = data_clim_period.groupby('valid_time.month').mean('valid_time')
    climatology_std = data_clim_period.groupby('valid_time.month').std('valid_time')

    # Replace zero standard deviation with a small number
    climatology_std = climatology_std.where(climatology_std > 0, other=1e-10)

    # Step 3: Calculate standardized anomalies for all valid_time in the original dataset
    data_full = ds[variable]
    standardized_anomalies = xr.apply_ufunc(
        lambda x, m, s: (x - m) / s,
        data_full.groupby('valid_time.month'),
        climatology_mean,
        climatology_std,
        dask="allowed"
    )

    # Step 4: Add standardized anomalies to the original dataset
    ds_standardized_anomalies = ds.copy()
    ds_standardized_anomalies[f"{variable}_std_anomaly"] = standardized_anomalies

    return ds_standardized_anomalies

In [17]:
date_range = ("1991-01-01", "2020-12-31")

ds = calculate_standardized_anomalies(ds, variable='tp', date_range=date_range)
ds = calculate_standardized_anomalies(ds, variable='t2m', date_range=date_range)
ds = calculate_standardized_anomalies(ds, variable='pev', date_range=date_range)
ds = calculate_standardized_anomalies(ds, variable='swvl1', date_range=date_range)

#### Land only

In [18]:
import xarray as xr

def create_land_variable(ds: xr.Dataset, variable: str, lsm_var: str = "lsm") -> xr.Dataset:
    """
    Create a land-only version of a specified variable using a land-sea mask (lsm).

    Args:
        ds (xr.Dataset): The input xarray dataset containing the variables.
        variable (str): The name of the variable to create the land-only version for.
        lsm_var (str): The name of the land-sea mask variable. Default is 'lsm'.

    Returns:
        xr.Dataset: A new dataset with the land-only version of the specified variable.

    Raises:
        ValueError: If the specified variable or land-sea mask is not present in the dataset.
    """
    if variable not in ds:
        raise ValueError(f"Variable '{variable}' not found in the dataset.")

    if lsm_var not in ds:
        raise ValueError(f"Land-sea mask variable '{lsm_var}' not found in the dataset.")

    # Ensure the land-sea mask is binary-like
    lsm = ds[lsm_var].where(ds[lsm_var] >= 0, other=0)

    # Mask the variable using the land-sea mask (threshold for land > 0.5)
    land_variable = ds[variable].where(lsm > 0.5)

    # Add the new variable to the dataset with a descriptive name
    land_var_name = f"{variable}_land_only"
    ds[land_var_name] = land_variable

    # Add metadata to the new variable
    ds[land_var_name].attrs.update({
        "description": f"Land-only version of {variable} based on {lsm_var}",
        "units": ds[variable].attrs.get("units", "unknown"),
        "land_only": True
    })

    return ds


In [19]:
# Example usage:
ds = create_land_variable(ds, "tp_std_anomaly")
ds = create_land_variable(ds, "t2m_std_anomaly")

ds = create_land_variable(ds, "tp_percent_of_normal")
ds = create_land_variable(ds, "t2m_percent_of_normal")

#### Barley crop mask

In [20]:
import xarray as xr
import os
import glob

# Define the preprocess function
def preprocess(file_path):
    """
    Preprocess a single dataset:
    - Load the dataset.
    - Extract the crop name from the file path.
    - Keep only the 'croparea' variable.
    - Rename 'croparea' to include the crop name.
    - Remove attributes to avoid conflicts.

    Args:
        file_path (str): Path to the NetCDF file.

    Returns:
        xarray.Dataset: Preprocessed dataset.
    """
    # Load the dataset
    ds = xr.open_dataset(file_path, chunks="auto")
    
    # Extract crop name from filename
    file_name = os.path.basename(file_path)
    crop_name = file_name.split("_")[-1].split(".")[0]  # Extracts 'carrot'

    # Keep only 'croparea' and rename it
    ds = ds[['croparea']].rename_vars({'croparea': f'{crop_name}_croparea'})

    # Remove conflicting attributes
    ds.attrs = {}
    return ds

# Define the directory containing your NetCDF files
data_dir = '/teamspace/studios/this_studio/drought-forecasting/modeling-pipeline/data/01_raw/unzipped_files/CROPGRIDS/CROPGRIDSv1.08_NC_maps/CROPGRIDSv1.08_NC_maps/'

# Create a sorted list of file paths
file_list = sorted(glob.glob(data_dir + '*.nc'))

# Exclude specific file
excluded_file = 'Countries_2018.nc'
file_list = [file for file in file_list if not file.endswith(excluded_file)]

# Preprocess each file and combine them
datasets = [preprocess(file_path) for file_path in file_list]
crop_ds = xr.combine_by_coords(datasets, combine_attrs="override")

In [21]:
# Apply mask: Set values below 100 to 0, and values equal to or above 100 to 1
crop_ds = crop_ds.map(lambda da: xr.where(da >= 1, 1, 0))

# Step 1: Adjust longitude for crop_ds
crop_ds = crop_ds.assign_coords(lon=((crop_ds.lon + 360) % 360)).sortby('lon')

In [22]:
import xarray as xr

def mask_with_crop_area(ds: xr.Dataset, crop_ds: xr.Dataset, variable: str, crop: str) -> xr.Dataset:
    """
    Interpolates `crop_ds` to match `ds` using Dask, then masks a variable in `ds` with a crop area from `crop_ds`.
    Creates a new variable in `ds` with the masked values.

    Args:
        ds (xr.Dataset): Dataset containing the variable to mask.
        crop_ds (xr.Dataset): Dataset containing crop area variables.
        variable (str): Name of the variable in `ds` to mask.
        crop (str): Specific crop variable to use for masking (e.g., 'cotton_croparea').

    Returns:
        xr.Dataset: Dataset with a new variable containing the masked values.

    Raises:
        ValueError: If the specified variable or crop is not found in the datasets.
    """
    if variable not in ds:
        raise ValueError(f"Variable '{variable}' not found in the dataset `ds`.")

    # Unify chunks for consistent Dask operations
    ds = ds.unify_chunks()
    crop_ds = crop_ds.unify_chunks()

    # Check if the specific crop is in the crop dataset
    if crop not in crop_ds:
        raise ValueError(f"Crop variable '{crop}' not found in the dataset `crop_ds`.")

    # Use the specific crop area as the mask
    mask = crop_ds[crop]

    # Interpolate crop mask to match `ds` spatial resolution using Dask
    interpolated_mask = mask.interp(
        lat=ds.latitude, lon=ds.longitude, method="nearest"
    )

    # Mask the variable
    masked_variable = ds[variable].where(interpolated_mask > 0)

    # Add the masked variable as a new variable in the dataset
    masked_variable_name = f"{variable}_{crop}"
    ds[masked_variable_name] = masked_variable

    return ds


In [23]:
crop_ds

<xarray.Dataset> Size: 36GB
Dimensions:                  (lat: 3600, lon: 7200)
Coordinates:
  * lat                      (lat) float32 14kB -89.97 -89.93 ... 89.93 89.97
  * lon                      (lon) float32 29kB 0.02499 0.07501 ... 359.9 360.0
Data variables: (12/173)
    abaca_croparea           (lat, lon) int64 207MB dask.array<chunksize=(3600, 7200), meta=np.ndarray>
    agave_croparea           (lat, lon) int64 207MB dask.array<chunksize=(3600, 7200), meta=np.ndarray>
    alfalfa_croparea         (lat, lon) int64 207MB dask.array<chunksize=(3600, 7200), meta=np.ndarray>
    almond_croparea          (lat, lon) int64 207MB dask.array<chunksize=(3600, 7200), meta=np.ndarray>
    aniseetc_croparea        (lat, lon) int64 207MB dask.array<chunksize=(3600, 7200), meta=np.ndarray>
    apple_croparea           (lat, lon) int64 207MB dask.array<chunksize=(3600, 7200), meta=np.ndarray>
    ...                       ...
    vetch_croparea           (lat, lon) int64 207MB dask.array<chunksize=(3600, 7200), meta=np.ndarray>
    walnut_croparea          (lat, lon) int64 207MB dask.array<chunksize=(3600, 7200), meta=np.ndarray>
    watermelon_croparea      (lat, lon) int64 207MB dask.array<chunksize=(3600, 7200), meta=np.ndarray>
    wheat_croparea           (lat, lon) int64 207MB dask.array<chunksize=(3600, 7200), meta=np.ndarray>
    yam_croparea             (lat, lon) int64 207MB dask.array<chunksize=(3600, 7200), meta=np.ndarray>
    yautia_croparea          (lat, lon) int64 207MB dask.array<chunksize=(3600, 7200), meta=np.ndarray>

To address the impacts of drought on agriculture, this study will focus on maize cultivation, utilizing a maize crop mask, as maize is the most widely grown staple crop in sub-Saharan Africa, particularly in East Africa, where it plays a critical role in food security due to its extensive cultivation and caloric contribution (FAO, 2021; Cairns et al., 2013).

**Sources**:

1. Food and Agriculture Organization (FAO). (2021). *The State of Food and Agriculture*. Rome: FAO. 
2. Cairns, J. E., Hellin, J., Sonder, K., Araus, J. L., MacRobert, J. F., Thierfelder, C., & Prasanna, B. M. (2013). Adapting maize production to climate change in sub-Saharan Africa. *Food Security, 5*(3), 345–360. [https://doi.org/10.1007/s12571-013-0256-x](https://doi.org/10.1007/s12571-013-0256-x)


In [24]:
# Apply mask for specific crop
ds = mask_with_crop_area(ds, crop_ds, "tp_percent_of_normal", crop="maize_croparea")
ds = mask_with_crop_area(ds, crop_ds, "tp_std_anomaly", crop="maize_croparea")

ds = mask_with_crop_area(ds, crop_ds, "t2m", crop="maize_croparea")
ds = mask_with_crop_area(ds, crop_ds, "t2m_percent_of_normal", crop="maize_croparea")
ds = mask_with_crop_area(ds, crop_ds, "t2m_std_anomaly", crop="maize_croparea")

ds = mask_with_crop_area(ds, crop_ds, "pev", crop="maize_croparea")
ds = mask_with_crop_area(ds, crop_ds, "pev_std_anomaly", crop="maize_croparea")

ds = mask_with_crop_area(ds, crop_ds, "swvl1", crop="maize_croparea")
ds = mask_with_crop_area(ds, crop_ds, "swvl1_percent_of_normal", crop="maize_croparea")
ds = mask_with_crop_area(ds, crop_ds, "swvl1_std_anomaly", crop="maize_croparea")

In [25]:
ds

<xarray.Dataset> Size: 103GB
Dimensions:                                 (valid_time: 551, latitude: 721,
                                             longitude: 1440)
Coordinates:
    number                                  int64 8B 0
  * valid_time                              (valid_time) datetime64[ns] 4kB 1...
  * latitude                                (latitude) float64 6kB 90.0 ... -...
  * longitude                               (longitude) float64 12kB 0.0 ... ...
    expver                                  (valid_time) <U4 9kB dask.array<chunksize=(45,), meta=np.ndarray>
    month                                   (valid_time) int64 4kB 1 2 ... 10 11
    lat                                     (latitude) float64 6kB 90.0 ... -...
    lon                                     (longitude) float64 12kB 0.0 ... ...
Data variables: (12/45)
    cl                                      (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(45, 121, 240), meta=np.ndarray>
    cvh                                     (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(45, 121, 240), meta=np.ndarray>
    tvh                                     (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(45, 121, 240), meta=np.ndarray>
    asn                                     (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(45, 121, 240), meta=np.ndarray>
    sst                                     (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(45, 121, 240), meta=np.ndarray>
    slt                                     (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(45, 121, 240), meta=np.ndarray>
    ...                                      ...
    t2m_std_anomaly_maize_croparea          (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(45, 121, 240), meta=np.ndarray>
    pev_maize_croparea                      (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(45, 121, 240), meta=np.ndarray>
    pev_std_anomaly_maize_croparea          (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(45, 121, 240), meta=np.ndarray>
    swvl1_maize_croparea                    (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(45, 121, 240), meta=np.ndarray>
    swvl1_percent_of_normal_maize_croparea  (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(45, 121, 240), meta=np.ndarray>
    swvl1_std_anomaly_maize_croparea        (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(45, 121, 240), meta=np.ndarray>
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts

#### Save dataset for ESTDA

In [26]:
# Convert dataset attributes
for attr, value in ds.attrs.items():
    if isinstance(value, bool):
        ds.attrs[attr] = str(value)

# Convert variable-specific attributes
for var_name, var in ds.variables.items():
    for attr, value in var.attrs.items():
        if isinstance(value, bool):
            var.attrs[attr] = str(value)

# Save the dataset
ds.to_netcdf('drought-forecasting/modeling-pipeline/data/02_intermediate/estda.nc')